In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q /content/drive/MyDrive/face_age.zip

In [ ]:
!nvidia-smi


In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("GPU is set and ready!")
    except RuntimeError as e:

        print(e)

In [ ]:
import numpy as np
import os
import pandas as pd
import kagglehub
import tensorflow as tf
import cv2
import imghdr
from tensorflow.keras.metrics import Precision, Recall, Accuracy
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Add
from tensorflow.keras.layers import Activation, GlobalAveragePooling2D, Dropout

In [ ]:
tf.random.set_seed(42)

In [ ]:
data_dir = 'face_age'

In [ ]:
image_exts = ["jpeg","jpg","bmp","png"]


In [ ]:
for image_class in os.listdir(data_dir):
    class_path = os.path.join(data_dir, image_class)
    if not os.path.isdir(class_path):
        continue

    for image in os.listdir(class_path):
        image_path = os.path.join(class_path, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)

            if tip not in image_exts:
                print(f"Not a valid image: {image_path}")
                os.remove(image_path)
        except Exception as e:
            print(f"Error processing {image_path}: {e}")

# **1. Load Data**

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

In [ ]:
type(data)

In [ ]:
for image, label in data.take(1):
    print("Image shape:", image.shape)
    print("Label:", label)

In [ ]:
import glob
for ds_store_file in glob.glob(f"{data_dir}/**/.DS_Store", recursive=True):
    os.remove(ds_store_file)

In [ ]:

class_names = sorted(os.listdir(data_dir))
age_labels = [int(name) for name in class_names]  # Convert subdir names to integers

# lookup table for class index → actual age
age_lookup = tf.constant(age_labels, dtype=tf.int32)

# 3. Map dataset labels to actual ages
data = tf.keras.utils.image_dataset_from_directory(data_dir)
data = data.map(lambda x, y: (x, tf.gather(age_lookup, y)))

# 4. Now apply your age-to-category mapping
def label_to_category(image, label):
    category = tf.where(
        label < 13, 0,
        tf.where(
            label < 20, 1,
            tf.where(label < 60, 2, 3)
        )
    )
    return image, tf.cast(category, tf.int32)

dataset = data.map(label_to_category)


In [ ]:
for image, label in dataset.take(1):
    print("Image shape:", image.shape)
    print("Label:", label)

## **2. Preprocess data**

###  2.1 Scaling the data

In [ ]:
data = data.map(lambda x,y:(x/255,y))


In [ ]:
scaled_iterator = data.as_numpy_iterator()

In [ ]:
batch = scaled_iterator.next()

In [ ]:
batch[1].shape

In [ ]:
dataset = dataset.map(lambda x,y:(x/255,y))

In [ ]:
scaled_iterator_1 = dataset.as_numpy_iterator()

In [ ]:
batch_dataset = scaled_iterator_1.next()

In [ ]:

print("Pixel min:", batch[0].min())
print("Pixel max:", batch[0].max())


In [ ]:

print("Pixel min:", batch_dataset[0].min())
print("Pixel max:", batch_dataset[0].max())


In [ ]:
batch_dataset[1].shape

In [ ]:
len(batch_dataset)

In [ ]:
fix , ax = plt.subplots(ncols = 4,figsize = (20,20))
for indx,img in enumerate(batch_dataset[0][:4]):
    ax[indx].imshow((img * 255).astype("uint8"))
    ax[indx].title.set_text(f"Label: {batch[1][indx]}")


### **2.2 Split**

In [ ]:
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)
test_size = len(data) - train_size - val_size  # 10% for testing

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

In [ ]:
length = dataset.cardinality().numpy()

train_size_d = int(length * 0.7)
val_size_d = int(length * 0.2)
test_size_d = length - train_size_d - val_size_d

train_dataset = dataset.take(train_size_d)
val_dataset = dataset.skip(train_size_d).take(val_size_d)
test_dataset = dataset.skip(train_size_d + val_size_d)

## **3 DeepLearing MODEL(CLassification)**

### **3.1 Build neural network(Classification Task)**

In [ ]:
def build_simple_cnn(
    input_shape=(256, 256, 3),
    hidden_layers=2,
    filters=[32, 16],
    kernel_size=(3, 3),
    activation='relu',
    include_pooling=True,
    fc_layers=[256],
    output_classes=4,
    use_skip_connections=False
):
    inputs = Input(shape=input_shape)
    x = Conv2D(16, kernel_size, padding='same', activation=activation)(inputs)
    if include_pooling:
        x = MaxPooling2D()(x)

    prev = x  # to store for skip connection

    for i in range(hidden_layers * 2):
        filter_idx = i % len(filters)
        current_filter = filters[filter_idx]

        conv = Conv2D(current_filter, kernel_size, padding='same', activation=activation)(x)

        if include_pooling:
            conv = MaxPooling2D()(conv)

        if use_skip_connections and conv.shape == prev.shape:
            x = Add()([conv, prev])  # skip connection
        else:
            x = conv

        prev = x

    # Flatten
    x = Flatten()(x)

    # Fully Connected Layers
    for fc_size in fc_layers:
        x = Dense(fc_size, activation=activation)(x)

    outputs = Dense(output_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model



### **Model-Default ReLU; Loss: sparse_categorical_crossentropy; Optimizer: Adam**

In [ ]:
# Default model - 3x3 kernels, relu activation, with pooling, 2 hidden layers
model_default = build_simple_cnn()



### **Tan-h Activation**

In [ ]:
# Changed activation to tanh
model_1 = build_simple_cnn(activation='tanh')

### **Pooling is removed**

In [ ]:
# No pooling layers
model_2 = build_simple_cnn(include_pooling=False)

### **5 X 5 kernels**

In [ ]:
# 5x5 kernels and hidden layer -  1(network depth = 2)
model_3 = build_simple_cnn(hidden_layers=2, kernel_size=(5, 5))

In [ ]:
# Combine multiple customization and hidden layer -  1(network depth = 1)
model_4 = build_simple_cnn(
    hidden_layers=1,
    activation='tanh',
    filters=[64, 32],
    include_pooling=False) # Reduce pooling in model_4 to prevent output size from becoming negative

In [ ]:
model_5= build_simple_cnn(
    hidden_layers=2,
    filters=[64, 32],
    use_skip_connections=True,         # Skip connections ON
    fc_layers=[256, 128],              # Two fully connected layers
    activation='relu',
    include_pooling=True
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping callback
early_stop = EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)


## **3.2 Train**

In [ ]:
model_default_hist = model_default.fit(train_dataset,epochs = 20,validation_data = val_dataset)

In [ ]:
hist_1 = model_1.fit(
    train_dataset,
    epochs=20,
    validation_data=val_dataset
)

In [ ]:
hist_2 = model_2.fit(train_dataset,epochs = 20,validation_data = val_dataset)

In [ ]:
hist_3 = model_3.fit(train_dataset,epochs = 20,validation_data = val_dataset)

In [ ]:
hist_4 = model_4.fit(train_dataset,epochs = 15,validation_data = val_dataset,callbacks=[early_stop])

In [ ]:
hist_5 = model_5.fit(train_dataset,epochs=15,validation_data=val_dataset)

###  **3.3 PLOT Performance**

In [ ]:
import plotly.graph_objs as go

def plot_model_losses_interactive_individual(histories, labels, title='Model Loss Comparison'):
    for hist, label in zip(histories, labels):
        fig = go.Figure()

        fig.add_trace(go.Scatter(
            y=hist.history['loss'],
            mode='lines',
            name='Train Loss'
        ))

        fig.add_trace(go.Scatter(
            y=hist.history['val_loss'],
            mode='lines',
            name='Validation Loss',
            line=dict(dash='dash')
        ))

        fig.update_layout(
            title=f'{title} - {label}',
            xaxis_title='Epoch',
            yaxis_title='Loss',
            hovermode='x unified',
            template='plotly_white'
        )

        fig.show()


In [ ]:
plot_model_losses_interactive_individual(
    histories=[model_default_hist, hist_1, hist_2, hist_3, hist_4,hist_5],
    labels=['Default', 'Model 1', 'Model 2', 'Model 3', 'Model 4','Model 5']
)

## **4. DeepLearing MODEL(Regression)**

### **4.1 Build neural network(Regression Task)**

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Add
)

def build_simple_reg_cnn(
    input_shape=(256, 256, 3),
    hidden_layers=2,
    filters=[32, 16],
    kernel_size=(3, 3),
    activation='relu',
    include_pooling=True,
    fc_size=256,
    output_classes=1,
    use_skip_connections=False
):
    inputs = Input(shape=input_shape)
    x = Conv2D(16, kernel_size, strides=1, padding='same', activation=activation)(inputs)
    if include_pooling:
        x = MaxPooling2D()(x)

    prev = x  # For skip connections

    for i in range(hidden_layers * 2):
        filter_idx = i % len(filters)
        current_filter = filters[filter_idx]

        conv = Conv2D(current_filter, kernel_size, padding='same', activation=activation)(x)

        if include_pooling:
            conv = MaxPooling2D()(conv)

        if use_skip_connections and conv.shape == prev.shape:
            x = Add()([conv, prev])
        else:
            x = conv

        prev = x

    x = Flatten()(x)
    x = Dense(fc_size, activation=activation)(x)
    x = Dense(fc_size // 2, activation=activation)(x)  # Second FC layer
    outputs = Dense(output_classes, activation='linear')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    return model


model_reg_default = build_simple_reg_cnn()
# Tanh activation
model_reg_1 = build_simple_reg_cnn(activation='tanh')

# No pooling layers
model_reg_2 = build_simple_reg_cnn(include_pooling=False, hidden_layers=1, filters=[16, 8])

# 5x5 kernels
model_reg_3 = build_simple_reg_cnn(kernel_size=(5, 5))

# Combination: tanh, no pooling, deeper filters
model_reg_4 = build_simple_reg_cnn(
    hidden_layers=2,
    activation='tanh',
    filters=[64, 32]
)

# Regression model with skip connections
model_reg_skip = build_simple_reg_cnn(
    use_skip_connections=True,
    filters=[64, 32],
    hidden_layers=2,
    include_pooling=True
)


## **4.2 Train**

In [ ]:
model_default_hist_reg = model_reg_default.fit(train,epochs = 20,validation_data = val)


In [ ]:
model_1_hist = model_reg_1.fit(train,epochs = 20,validation_data = val)

In [ ]:
model_2_hist = model_reg_2.fit(train,epochs = 20,validation_data = val)

In [ ]:
model_3_hist = model_reg_3.fit(train,epochs = 20,validation_data = val,callbacks=[early_stop])

In [ ]:
model_4_hist = model_reg_4.fit(train,epochs = 20,validation_data = val)

In [ ]:
model_reg_skip_hist = model_reg_skip.fit(train,epochs = 20,validation_data = val)

###  **4.3 PLOT Performance**

In [ ]:
def plot_model_losses(histories, labels, title='Model Loss Comparison'):
    plt.figure(figsize=(10, 6))

    for hist, label in zip(histories, labels):
        plt.plot(hist.history['loss'], label=f'{label} - Train')
        plt.plot(hist.history['val_loss'], linestyle='--', label=f'{label} - Val')

    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()


In [ ]:
plot_model_losses(
    histories=[model_default_hist_reg, model_1_hist, model_2_hist, model_3_hist ,model_4_hist,model_reg_skip_hist],
    labels=['Default', 'Model 1', 'Model 2', 'Model 3', 'Model 4']
)

### **5.Testing(Classification)**



In [ ]:
def evaluate_classification_model(model, test_dataset):
    pre = Precision()
    re = Recall()
    acc = Accuracy()

    for batch in test_dataset.as_numpy_iterator():
        x, y = batch
        yhat = model.predict(x, verbose=0)

        # Get true and predicted class indices
        y_true = y.argmax(axis=1) if y.ndim > 1 else y
        y_pred = yhat.argmax(axis=1)

        # Update metrics
        pre.update_state(y_true, y_pred)
        re.update_state(y_true, y_pred)
        acc.update_state(y_true, y_pred)

    precision = pre.result().numpy()
    recall = re.result().numpy()
    accuracy = acc.result().numpy()
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-8)  # Avoid division by zero

    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1-score: {f1_score:.4f}")

    return {
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy,
        "f1_score": f1_score
    }


In [ ]:

model_default_test = evaluate_classification_model(model_default, test_dataset)


In [ ]:

model_1_test = evaluate_classification_model(model_1, test_dataset)

In [ ]:
model_2_test = evaluate_classification_model(model_2, test_dataset)

In [ ]:

model_3_test = evaluate_classification_model(model_3, test_dataset)

In [ ]:

model_4_test = evaluate_classification_model(model_4, test_dataset)

### **6.Testing(Regression)**



In [ ]:
def evaluate_models_on_test(models, model_names, test_dataset):
    print(f"{'Model':<20} {'Test Loss (MSE)':<20} {'Test MAE':<15}")
    print("-" * 55)
    for model, name in zip(models, model_names):
        test_loss, test_mae = model.evaluate(test_dataset, verbose=0)
        print(f"{name:<20} {test_loss:<20.4f} {test_mae:<15.4f}")


In [ ]:
evaluate_models_on_test(
    models=[model_reg_default, model_reg_1, model_reg_2, model_reg_3, model_reg_4, model_reg_skip],  # Use model objects instead of History objects
    model_names=["Default", "Model 1", "Model 2", "Model 3", "Model 4", "Model 5"],
    test_dataset=test
)

In [ ]:
from tensorflow import keras

# Save regression model
model_reg_skip.save('model_reg_skip.keras')

# Save classification model
model_1.save('model_1.keras')


In [ ]:
import os
import shutil

os.makedirs("my_models", exist_ok=True)
shutil.move("model_reg_skip.keras", "my_models/model_reg_skip.keras")
shutil.move("model_1.keras", "my_models/model_1.keras")
shutil.make_archive("my_models", 'zip', "my_models")
